In [132]:
import numpy as np
from typing import List, Optional, Tuple

# ==========================================================
# 1. Scratchpad Memory (SPM) - handles tiling and padding
# ==========================================================
class ScratchpadMemory:
    def __init__(self, input_tensor, tile_size, kernel_size, stride=1, verbose=True):
        self.input = input_tensor
        self.H, self.W, self.C = input_tensor.shape
        self.tile_size = tile_size
        self.kernel_size = kernel_size
        self.stride = stride
        self.verbose = verbose

    def generate_tile_addresses(self):
        T, K, s = self.tile_size, self.kernel_size, self.stride
        addrs = []
        row_step = T - K + 1 if T - K + 1 > 0 else 1
        col_step = T - K + 1 if T - K + 1 > 0 else 1

        num_tiles_row = int(np.ceil((self.H - K + 1) / row_step))
        num_tiles_col = int(np.ceil((self.W - K + 1) / col_step))

        if self.verbose:
            print(f"\n[SPM] Generating tile addresses...")
            print(f"  Input shape = ({self.H},{self.W},{self.C}), Tile size = {T}, Kernel = {K}, Stride = {s}")
            print(f"  Step sizes -> row_step = {row_step}, col_step = {col_step}")
            print(f"  Expected total tiles = {num_tiles_row} x {num_tiles_col} = {num_tiles_row*num_tiles_col}\n")

        for t_i in range(num_tiles_row):
            for t_j in range(num_tiles_col):
                row = t_i * row_step
                col = t_j * col_step
                addrs.append((row, col))
                if self.verbose:
                    pad_r = max(0, row + T - self.H)
                    pad_c = max(0, col + T - self.W)
                    print(f"  -> Tile ({t_i},{t_j}): start=({row},{col}), "
                          f"cover=({row}:{row+T}, {col}:{col+T}), "
                          f"pad_rows={pad_r}, pad_cols={pad_c}")

        return addrs

    def read_row(self, tile_start, row_idx):
        r0, c0 = tile_start
        T = self.tile_size
        row_data = np.zeros((T, self.C))
        for j in range(T):
            rr, cc = r0 + row_idx, c0 + j
            if rr < self.H and cc < self.W:
                row_data[j, :] = self.input[rr, cc, :]
        if self.verbose:
            print(f"[SPM] Read row {row_idx} from tile starting at {tile_start}, "
                  f"valid range=({r0+row_idx},{c0}:{c0+T})")
        return row_data  # shape [T, C]

In [133]:
class ToeplitzBuffer:
    def __init__(self, tile_size, kernel_size, num_channels, stride=1):
        self.tile_size = tile_size
        self.kernel_size = kernel_size
        self.num_channels = num_channels
        self.stride = stride
        self.buffer = []

    def stream_row(self, row_data):
        """
        row_data: shape [tile_size, num_channels]
        Returns: list of Toeplitz rows (flattened) ready for SA
        """
        self.buffer.append(row_data)
        if len(self.buffer) < self.kernel_size:
            return []

        K = self.kernel_size
        C = self.num_channels
        toeplitz_rows = []

        current_block = np.stack(self.buffer[-K:], axis=0)  # shape [K, tile_size, C]

        for j in range(0, self.tile_size - K + 1, self.stride):
            patch = current_block[:, j:j+K, :].reshape(-1)
            toeplitz_rows.append(patch)

        return toeplitz_rows



In [134]:
class KernelLoader:
    def __init__(self, kernels: np.ndarray, array_size: int):
        self.R, self.S, self.C, self.K = kernels.shape
        self.kernels = kernels
        self.array_size = array_size

    def get_kernel_matrix(self, k: int):
        """
        Flatten one kernel and return padded [array_size, array_size] matrix.
        """
        k_flat = self.kernels[:, :, :, k].reshape(-1)  # shape [R*S*C]
        H = len(k_flat)
        padded = np.zeros((self.array_size, self.array_size))
        padded[:H, 0] = k_flat  # map into first column (common for weight-stationary)
        return padded


In [135]:
# # ==========================================================
# # 4. Processing Element (PE)
# # ==========================================================
# class PE:
#     def __init__(self, links: Optional[List['PE']] = None):
#         if links is None:
#             links = [None, None, None, None]
#         self.links = links
#         self.activation = 0.0
#         self.weight = 0.0
#         self.accumulation = 0.0
    
#     def shift(self, shift_direction: int, activation_flag: bool):
#         neighbor = self.links[shift_direction]
#         if neighbor is not None:
#             if activation_flag:
#                 neighbor.activation = self.activation
#             else:
#                 neighbor.weight = self.weight

In [136]:
# class SystolicArray:
#     def __init__(self, size: int):
#         self.size = size
#         self.array = self._setup_array()
    
#     def _setup_array(self):
#         array = [[PE() for _ in range(self.size)] for _ in range(self.size)]
#         for i in range(self.size):
#             for j in range(self.size):
#                 left = array[i][j-1] if j > 0 else None
#                 up = array[i-1][j] if i > 0 else None
#                 right = array[i][j+1] if j < self.size-1 else None
#                 down = array[i+1][j] if i < self.size-1 else None
#                 array[i][j].links = [left, up, right, down]
#         return array

#     def load_weights(self, weight_matrix):
#         # weight_matrix shape: [size, size]
#         for i in range(self.size):
#             for j in range(self.size):
#                 self.array[i][j].weight = weight_matrix[i, j]

#     def feed_activation_row(self, row):
#         # row length should be <= self.size
#         for j in range(len(row)):
#             self.array[0][j].activation = row[j]

#     def cycle(self):
#         # compute accumulation
#         for i in range(self.size):
#             for j in range(self.size):
#                 pe = self.array[i][j]
#                 pe.accumulation += pe.activation * pe.weight
#         # shift activations down and weights right
#         for i in reversed(range(self.size)):
#             for j in reversed(range(self.size)):
#                 pe = self.array[i][j]
#                 if i < self.size - 1:
#                     self.array[i+1][j].activation = pe.activation
#                 if j < self.size - 1:
#                     self.array[i][j+1].weight = pe.weight

#     def collect_output(self):
#         out = np.zeros((self.size, self.size))
#         for i in range(self.size):
#             for j in range(self.size):
#                 out[i,j] = self.array[i][j].accumulation
#         return out

#     def reset_accumulation(self):
#         for i in range(self.size):
#             for j in range(self.size):
#                 self.array[i][j].accumulation = 0.0

In [137]:
class PE:
    def __init__(self, links: Optional[List['PE']] = None):
        if links is None:
            links = [None, None, None, None]
        self.links = links
        self.activation = 0.0
        self.weight = 0.0
        self.accumulation = 0.0

class SystolicArray:
    def __init__(self, size: int):
        self.size = size
        self.array = [[PE() for _ in range(size)] for _ in range(size)]

    def load_weights(self, weight_matrix):
        for i in range(self.size):
            for j in range(self.size):
                self.array[i][j].weight = weight_matrix[i,j]

    def feed_activation_row(self, row):
        for j in range(len(row)):
            self.array[0][j].activation = row[j]

    def cycle(self):
        # compute accumulation
        for i in range(self.size):
            for j in range(self.size):
                pe = self.array[i][j]
                pe.accumulation += pe.activation * pe.weight
        # shift activations down
        for i in reversed(range(self.size-1)):
            for j in range(self.size):
                self.array[i+1][j].activation = self.array[i][j].activation

    def collect_output(self):
        out = np.zeros((self.size, self.size))
        for i in range(self.size):
            for j in range(self.size):
                out[i,j] = self.array[i][j].accumulation
        return out

    def reset_accumulation(self):
        for i in range(self.size):
            for j in range(self.size):
                self.array[i][j].accumulation = 0.0

In [138]:
# ==========================================================
# 6. Accumulator
# ==========================================================
class PsumBuffer:
    def __init__(self):
        self.outputs = []

    def accumulate(self, tile_output):
        self.outputs.append(tile_output)

    def get_final(self):
        return np.sum(self.outputs, axis=0)

In [139]:
# class ConvEngine:
#     def __init__(self, input_tensor, kernels, array_size=32, stride=1, verbose=True):
#         self.input_tensor = input_tensor  # [H, W, C]
#         self.kernels = kernels            # [R, S, C, K]
#         self.array_size = array_size
#         self.stride = stride
#         self.verbose = verbose

#         self.H, self.W, self.C = input_tensor.shape
#         self.R, self.S, self.Ck, self.K = kernels.shape
#         assert self.C == self.Ck, "Channel mismatch!"

#         self.sa = SystolicArray(array_size)

#     def run(self):
#         loader = KernelLoader(self.kernels, self.array_size)
#         spm = ScratchpadMemory(self.input_tensor, self.array_size, self.R, stride=self.stride, verbose=self.verbose)

#         # Output dimensions
#         out_H = (self.H - self.R) // self.stride + 1
#         out_W = (self.W - self.S) // self.stride + 1
#         output = np.zeros((out_H, out_W, self.K))

#         # 1. Get all tile start addresses
#         tile_addrs = spm.generate_tile_addresses()

#         # 2. Loop over kernels
#         for k in range(self.K):
#             if self.verbose:
#                 print(f"\n[Engine] === Loading Kernel {k} into SA ===")
#             weight_matrix = loader.get_kernel_matrix(k)
#             self.sa.reset_accumulation()
#             self.sa.load_weights(weight_matrix)

#             # 3. Loop over tiles
#             for (row, col) in tile_addrs:
#                 if self.verbose:
#                     print(f"\n[Engine] Processing tile starting at ({row},{col}) for Kernel {k}")

#                 toeplitz = ToeplitzBuffer(self.array_size, self.R, self.C, stride=self.stride)

#                 # 4. Stream rows of tile into Toeplitz buffer
#                 for r in range(self.array_size):
#                     row_data = spm.read_row((row, col), r)  # shape [tile_size, C]
#                     toeplitz_rows = toeplitz.stream_row(row_data)

#                     # 5. Stream Toeplitz rows into SA
#                     for act_row in toeplitz_rows:
#                         if len(act_row) < self.array_size:
#                             act_row = np.pad(act_row, (0, self.array_size - len(act_row)))
#                         self.sa.feed_activation_row(act_row)
#                         self.sa.cycle()

#                 # 6. Collect psums from SA
#                 sa_out = self.sa.collect_output()
#                 block_H = min(self.array_size, out_H - row)
#                 block_W = min(self.array_size, out_W - col)
#                 output[row:row+block_H, col:col+block_W, k] = sa_out[:block_H, :block_W]

#         return output

In [140]:
# class ConvEngine:
#     def __init__(self, input_tensor, kernels, array_size=32, stride=1, verbose=True):
#         self.input_tensor = input_tensor
#         self.kernels = kernels
#         self.array_size = array_size
#         self.stride = stride
#         self.verbose = verbose

#         self.H, self.W, self.C = input_tensor.shape
#         self.R, self.S, self.Ck, self.K = kernels.shape
#         assert self.C == self.Ck, "Channel mismatch!"

#         self.sa = SystolicArray(array_size)

#     def run(self):
#         spm = ScratchpadMemory(self.input_tensor, self.array_size, self.R,
#                                stride=self.stride, verbose=self.verbose)

#         # Output dimensions
#         out_H = (self.H - self.R) // self.stride + 1
#         out_W = (self.W - self.S) // self.stride + 1
#         output = np.zeros((out_H, out_W, self.K))

#         # 1. Get all tile start addresses
#         tile_addrs = spm.generate_tile_addresses()
#         if self.verbose:
#             print(f"\n[Engine] === Tiling complete: {len(tile_addrs)} tiles ===")

#         loader = KernelLoader(self.kernels, self.array_size)

#         # 2. Process kernels one column at a time (weight-stationary)
#         for k in range(self.K):
#             k_flat = self.kernels[:, :, :, k].reshape(-1)  # R*S*C
#             num_chunks = int(np.ceil(len(k_flat) / self.array_size))

#             if self.verbose:
#                 print(f"\n[Engine] === Loading Kernel {k} into column {k % self.array_size} ===")

#             self.sa.reset_accumulation()

#             # Stream kernel chunks into the column
#             for chunk_idx in range(num_chunks):
#                 start = chunk_idx * self.array_size
#                 end = min((chunk_idx + 1) * self.array_size, len(k_flat))
#                 chunk_data = k_flat[start:end]

#                 if len(chunk_data) < self.array_size:
#                     chunk_data = np.pad(chunk_data, (0, self.array_size - len(chunk_data)))

#                 weight_matrix = np.zeros((self.array_size, self.array_size))
#                 col_idx = k % self.array_size  # assign column
#                 weight_matrix[:len(chunk_data), col_idx] = chunk_data

#                 if self.verbose:
#                     print(f"[Kernel {k}] Chunk {chunk_idx+1}/{num_chunks} loaded, "
#                         f"indices {start}:{end} into column {col_idx}")

#                 self.sa.load_weights(weight_matrix)

#             # 3. Loop over tiles
#             for (row, col) in tile_addrs:
#                 if self.verbose:
#                     print(f"\n[Engine] Processing tile at start=({row},{col}) for kernel {k}")

#                 toeplitz = ToeplitzBuffer(self.array_size, self.R, self.C, stride=self.stride)

#                 # 4. Stream rows into Toeplitz buffer → SA
#                 for r in range(self.array_size):
#                     row_data = spm.read_row((row, col), r)
#                     toeplitz_rows = toeplitz.stream_row(row_data)

#                     for act_row in toeplitz_rows:
#                         if len(act_row) < self.array_size:
#                             act_row = np.pad(act_row, (0, self.array_size - len(act_row)))
#                         self.sa.feed_activation_row(act_row)
#                         self.sa.cycle()

#                 # 5. Collect psums from SA
#                 sa_out = self.sa.collect_output()
#                 block_H = min(self.array_size, out_H - row)
#                 block_W = min(self.array_size, out_W - col)

#                 # 6. Stitch into outputs
#                 output[row:row+block_H, col:col+block_W, k] = sa_out[:block_H, col_idx:col_idx+block_W]

#         return output

In [141]:
class ConvEngine:
    def __init__(self, input_tensor, kernels, array_size=32, stride=1, verbose=True):
        self.input_tensor = input_tensor
        self.kernels = kernels
        self.array_size = array_size
        self.stride = stride
        self.verbose = verbose

        self.H, self.W, self.C = input_tensor.shape
        self.R, self.S, self.Ck, self.K = kernels.shape
        assert self.C == self.Ck, "Channel mismatch!"

        self.sa = SystolicArray(array_size)

    def run(self):
        spm = ScratchpadMemory(self.input_tensor, self.array_size, self.R,
                            stride=self.stride, verbose=self.verbose)

        # Output dimensions
        out_H = (self.H - self.R) // self.stride + 1
        out_W = (self.W - self.S) // self.stride + 1
        output = np.zeros((out_H, out_W, self.K))

        # 1. Get all tile start addresses
        tile_addrs = spm.generate_tile_addresses()
        if self.verbose:
            print(f"\n[Engine] === Tiling complete: {len(tile_addrs)} tiles ===")

        # 2. Process kernels in groups of SA width
        group_size = self.array_size
        num_groups = int(np.ceil(self.K / group_size))

        for g in range(num_groups):
            k_start = g * group_size
            k_end = min((g+1) * group_size, self.K)

            if self.verbose:
                print(f"\n[Engine] === Loading kernel group {g} ({k_start}..{k_end-1}) into SA ===")

            # Reset SA accumulators
            self.sa.reset_accumulation()

            # 2a. Load kernels into SA, one per column
            for col, k in enumerate(range(k_start, k_end)):
                k_flat = self.kernels[:, :, :, k].reshape(-1)  # Flattened kernel (R*S*C)

                # Break kernel into chunks of array_size
                num_chunks = int(np.ceil(len(k_flat) / self.array_size))
                for chunk_id in range(num_chunks):
                    start = chunk_id * self.array_size
                    end = min((chunk_id + 1) * self.array_size, len(k_flat))
                    chunk = np.zeros(self.array_size)
                    chunk[:end - start] = k_flat[start:end]

                    # Place chunk into column
                    weight_matrix = np.zeros((self.array_size, self.array_size))
                    weight_matrix[:, col] = chunk
                    self.sa.load_weights(weight_matrix)

                    if self.verbose:
                        print(f"[Kernel {k}] Chunk {chunk_id+1}/{num_chunks} loaded "
                            f"indices {start}:{end} into column {col}")

                    self.sa.cycle()  # settle weights

            # 3. Loop over tiles
            for (row, col) in tile_addrs:
                if self.verbose:
                    print(f"\n[Engine] Processing tile at start=({row},{col}) for kernels {k_start}..{k_end-1}")

                toeplitz = ToeplitzBuffer(self.array_size, self.R, self.C, stride=self.stride)

                # 4. Stream activation rows into Toeplitz buffer → SA
                for r in range(self.array_size):
                    row_data = spm.read_row((row, col), r)

                    # Toeplitz may produce multiple rows
                    toeplitz_rows = toeplitz.stream_row(row_data)

                    for act_row in toeplitz_rows:
                        # Break activation row into chunks of SA width
                        num_chunks = int(np.ceil(len(act_row) / self.array_size))
                        for chunk_id in range(num_chunks):
                            start = chunk_id * self.array_size
                            end = min((chunk_id + 1) * self.array_size, len(act_row))
                            chunk = np.zeros(self.array_size)
                            chunk[:end - start] = act_row[start:end]

                            self.sa.feed_activation_row(chunk)
                            self.sa.cycle()

                # 5. Collect psums from SA
                sa_out = self.sa.collect_output()
                block_H = min(self.array_size, out_H - row)
                block_W = min(self.array_size, out_W - col)

                # 6. Stitch into outputs for each kernel in group
                for col_idx, k in enumerate(range(k_start, k_end)):
                    output[row:row + block_H, col:col + block_W, k] = \
                        sa_out[:block_H, col_idx:col_idx + block_W]

        return output

In [142]:
if __name__ == "__main__":
    H, W, C = 64, 64, 3
    R = S = 3
    stride = 1
    input_tensor = np.random.randint(0,5,(H,W,C)).astype(float)
    kernels = np.random.randint(0,5,(R,S,C,1)).astype(float)

    engine = ConvEngine(input_tensor, kernels, stride=stride, array_size=32, verbose=True)
    sa_output = engine.run()

    # Golden convolution
    def conv2d_stride(input_tensor, kernel, stride=1):
        H, W, C = input_tensor.shape
        R, S, _, K = kernel.shape
        out_H = (H - R)//stride + 1
        out_W = (W - S)//stride + 1
        out = np.zeros((out_H, out_W, K))
        for k in range(K):
            for i in range(out_H):
                for j in range(out_W):
                    patch = input_tensor[i*stride:i*stride+R, j*stride:j*stride+S, :]
                    out[i,j,k] = np.sum(patch * kernel[:,:,:,k])
        return out

    golden = conv2d_stride(input_tensor, kernels, stride=stride)
    print("SA Simulator Output Shape:", sa_output.shape)
    print("Golden Conv Shape:", golden.shape)
    for k in range(sa_output.shape[2]):
        diff = np.max(np.abs(sa_output[:,:,k] - golden[:,:,k]))
        print(f"Kernel {k} max absolute difference: {diff}")


[SPM] Generating tile addresses...
  Input shape = (64,64,3), Tile size = 32, Kernel = 3, Stride = 1
  Step sizes -> row_step = 30, col_step = 30
  Expected total tiles = 3 x 3 = 9

  -> Tile (0,0): start=(0,0), cover=(0:32, 0:32), pad_rows=0, pad_cols=0
  -> Tile (0,1): start=(0,30), cover=(0:32, 30:62), pad_rows=0, pad_cols=0
  -> Tile (0,2): start=(0,60), cover=(0:32, 60:92), pad_rows=0, pad_cols=28
  -> Tile (1,0): start=(30,0), cover=(30:62, 0:32), pad_rows=0, pad_cols=0
  -> Tile (1,1): start=(30,30), cover=(30:62, 30:62), pad_rows=0, pad_cols=0
  -> Tile (1,2): start=(30,60), cover=(30:62, 60:92), pad_rows=0, pad_cols=28
  -> Tile (2,0): start=(60,0), cover=(60:92, 0:32), pad_rows=28, pad_cols=0
  -> Tile (2,1): start=(60,30), cover=(60:92, 30:62), pad_rows=28, pad_cols=0
  -> Tile (2,2): start=(60,60), cover=(60:92, 60:92), pad_rows=28, pad_cols=28

[Engine] === Tiling complete: 9 tiles ===

[Engine] === Loading kernel group 0 (0..0) into SA ===
[Kernel 0] Chunk 1/1 loaded ind